# Data importing, cleaning, and merging

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv('usc_report.csv', encoding = 'ISO-8859-1', low_memory=False)

In [4]:
df.columns

Index(['User_ID', 'Guest', 'Email_Address', 'Phone_Number', 'Pre_Resy_Diner',
       'First_Visit', 'Last_Visit', 'Completed_Reservations', 'Walk_in',
       'No_Show', 'Cancelled_Reservations', 'Brunch_Visits', 'Lunch_Visits',
       'Dinner_Visits', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'Wine Dinner Resevations', 'Average_Party_Size',
       'Tag', 'Guest_Notes', 'Reservations', 'Ticket_ID', 'Item',
       'Item_Category', 'quantity', 'Item_Price', 'Total_Check_Spend',
       'All_Time_Spend'],
      dtype='object')

In [5]:
#convert Pre_Resy_Diner to 0 or 1
df['Pre_Resy_Diner'] = df['Pre_Resy_Diner'].apply(lambda x: 0 if x=='New USC' else 1)

In [6]:
#Convert columns that should be numbers to int or float
#built-in Pandas methods don't work, hence custom code

def rem_non_nums(val):
    if type(val) == (int or float):
        return val
    else:
        try:
            return int(val)
        except:
            return 0

#run the number cleaning function on every numerical column
cols_that_should_be_ints = ['Completed_Reservations','Walk_in', 'No_Show', 'Cancelled_Reservations','Brunch_Visits','Lunch_Visits','Dinner_Visits','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Wine Dinner Resevations','Average_Party_Size','All_Time_Spend']

for col in cols_that_should_be_ints:
    df[col] = df[col].map(rem_non_nums)

In [7]:
len(df.loc[df['All_Time_Spend'] == 0])

27488

In [8]:
len(df.loc[df['Completed_Reservations'] == 0])

19870

## Import and merge email data

In [10]:
email = pd.read_csv('usc_mailing_list.csv', usecols=[0, 4, 5, 6, 16])
email = email.fillna('');

In [11]:
#create dummy variable for everyone who opted in for email - so that we can sort them later 
email['Opted_in'] = email['Email Address'].apply(lambda x: 1 if x !='' else 0)

#replace wine dinner updates categorical variable with dummy numerical variable
email['Want wine dinner updates?'] = email['Want wine dinner updates?'].apply(lambda x: 0 if x == '' else 1)

In [12]:
print(len(email), len(df))

66448 105562


In [13]:
df = pd.merge(df, email, how='left', left_on='Email_Address', right_on='Email Address')
len(df)

105562

## Move bottle price data to a unified column

In [14]:
#only keep item price column data when the price is for a bottle of wine
len(df.dropna()[df.dropna()['Item_Category'].str.contains('bottle', case=False)][['Item_Price']])

wine = df.dropna()[df.dropna()['Item_Category'].str.contains('bottle', case=False)][['Item_Price']]
wine.columns = ['bottle_price']

#merge the wine data back into the main dataframe
df = pd.merge(df, wine, how='left', left_index=True, right_index=True)

#check how many rows have wine purchase data
print(len(df[~df['bottle_price'].isnull()]), len(df))

869 105562


In [16]:
#convert NaN bottle prices to 0 before de-duping
df['bottle_price'] = df['bottle_price'].fillna(0)

#convert bottle prices to ints
df['bottle_price'] = df['bottle_price'].map(rem_non_nums);

#make the bottle price the sum of the bottle prices per user for the de-dupe
df['bottle_price'] = df.groupby('User_ID')['bottle_price'].transform('sum')

## De-dupe

In [17]:
print(len(df.drop_duplicates('User_ID')), len(df)) #length with no duplicates - target after de-duping

75195 105562


In [18]:
df = df.drop_duplicates(subset='User_ID')
len(df)

75195

In [19]:
df = df.fillna(0)

## Convert useful tags to dummy variables

In [20]:
#get a list of tags
#tags are raw strings, comma separated
tag_list = []
temp_arr = []

for tag_arr in df.Tag:
    temp_arr = tag_arr.split(',') #split string on commas
    temp_arr = [i.replace(' ','') for i in temp_arr] #remove spaces
    for tag in temp_arr:
        if tag not in tag_list:
            tag_list.append(tag)

tag_list.sort()
tag_list[:5]

['', '!Googled', '#betterlatethannever', '**ALERTBAR**', '**AlertKitchen**']

In [21]:
## create dummy variable for regulars
def is_reg(tags):
    if ('regular' or 'reg') in tags.lower():
        return 1
    else:
        return 0

df['Reg'] = df.Tag.apply(is_reg)
len(df.loc[df.Reg == 1])

1843

In [22]:
#create dummy variable for 'great next chapter'
def has_allergy(tags):
    if 'allergy' in tags.lower():
        return 1
    else:
        return 0
    
df['Allergy'] = df.Tag.apply(has_allergy)
len(df.loc[df.Allergy == 1])

1165

In [23]:
#create dummy variable for industry
def is_industry(tags):
    if 'industry' in tags.lower():
        return 1
    else:
        return 0
    
df['Industry'] = df.Tag.apply(is_industry)
len(df.loc[df.Industry == 1])

1999

In [24]:
#create dummy variable for soigne
def is_soigne(tags):
    if 'soigne' in tags.lower():
        return 1
    else:
        return 0
    
df['Soigne'] = df.Tag.apply(is_soigne)
len(df.loc[df.Soigne == 1])

1716

In [25]:
#create dummy variable for wine lovers
def is_wine_lover(tags):
    if 'wine' in tags.lower():
        return 1
    else:
        return 0
    
df['Wine_Lover'] = df.Tag.apply(is_wine_lover)
len(df.loc[df.Wine_Lover == 1])

1193

In [26]:
df.loc[df['Wine_Lover'] == 1]['All_Time_Spend'].mean()

810.2212908633696

In [27]:
df.loc[df['Wine_Lover'] != 1]['All_Time_Spend'].mean()

196.892921812924

**Calculate spend per visit**

In [28]:
df['spend_per_visit'] = df.All_Time_Spend/df.Completed_Reservations
df.spend_per_visit = df.spend_per_visit.fillna(0)

#get rid of inf
df.spend_per_visit = df.spend_per_visit.apply(lambda x: x if x < 10*100 else 0)
df.spend_per_visit.max()

998.0

## Drop to just columns we want

In [29]:
cols_to_drop = ['Guest','Email_Address','Phone_Number','First_Visit','Last_Visit','Tag','Guest_Notes','Reservations','Ticket_ID','Item','Item_Category','quantity','Item_Price','Total_Check_Spend','Email Address','OPTIN_TIME','REGION']
df = df.drop(cols_to_drop, axis=1)

In [31]:
df.to_csv(path_or_buf='cleaned_data.csv', index=False)